# <center>Fase 1 y 2: </center>
_____
______
## <span style="color:#f7e85f">**EDA + Limpieza de datos**</span>


###  📚 Importar <span style="color:#f7e85f">Librerías</span> y Comprobando <span style="color:#f7e85f">Visualizaciones</span>

In [1]:
# Importar librerías y extender visualización
import pandas as pd
import numpy as np
import re 

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg
import seaborn as sns

import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest, ttest_ind, norm, chi2_contingency, f_oneway, chi2_contingency, mannwhitneyu, kruskal, kendalltau

from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

import mysql.connector
from mysql.connector import errorcode

# -----------------------------------------------------------------------
# Gestión de los warnings
import warnings
warnings.filterwarnings("ignore")



# -----------------------------------------------------------------------
# Comprobando que el display sea el máximo para mostrar mejor el data frame
pd.set_option("display.max_rows", None)  # Cambia el número máximo de filas mostradas
pd.set_option("display.max_columns", None)  # Cambia el número máximo de columnas mostradas
pd.set_option("display.max_colwidth", None)  # Cambia el ancho máximo de la pantalla

###  📖 <span style="color:#f7e85f">Leer archivos</span>

In [2]:
df_clientes = pd.read_csv(r"C:\Users\yaelp\Desktop\Adalab\Proyectos grupo\Proyecto_04\clientes.csv")
df_productos = pd.read_csv(r"C:\Users\yaelp\Desktop\Adalab\Proyectos grupo\Proyecto_04\productos.csv")
df_ventas = pd.read_csv(r"C:\Users\yaelp\Desktop\Adalab\Proyectos grupo\Proyecto_04\ventas.csv")

### 🧼 <span style="color:#f7e85f">Limpieza de datos</span>

* Hacer cambios en <span style="color:#f7e85f">**df_productos**</span>
    * En la columna precio, hay que quitar el metacaracter de euros € y mandar a float
    * Hay 600 nulos en la columna precio

In [3]:
# Quitar metacaracter de euro € ---------------------------------------------------------------------------------

columnas1 = ["precio"]
def cambiar_simbolo(columnas):
    try:
        return columnas.str.replace("€","").str.replace(",", "").str.strip()
    except:
        return np.nan
for columna in columnas1:
    df_productos[columna] = cambiar_simbolo(df_productos[columna])

df_productos.head()
# Cambiar de objeto a float -------------------------------------------------------------------------------------
df_productos["precio"] = df_productos["precio"].astype(float)

df_productos.dtypes

id_producto     object
precio         float64
categoria       object
descripcion     object
dtype: object

* Hacer cambios en <span style="color:#f7e85f">**df_clientes**</span>
    * Crear columna nueva llamada "pais"
        * Traer después de la coma de ciudad todo
    * Editar columna ciudad
        * eliminar todo después de la coma

In [4]:
# Crear columna pais -------------------------------------------------------------------------------------------
df_clientes["pais"] = df_clientes['ciudad'].apply(lambda x: x.split(',')[-1].strip())
# Editar columna ciudad ----------------------------------------------------------------------------------------
df_clientes['ciudad'] = df_clientes['ciudad'].str.split(',').str[0]

* Guardando los <span style="color:#f7e85f">**CSVs Limpios**</span>

In [5]:
df_clientes.to_csv("clientes_limpio.csv")
df_ventas.to_csv("ventas_limpio.csv")
df_productos.to_csv("productos_limpio.csv")

# <center>**Bonus BBDD**: </center>
______
_____

### 🔗 Conectar crear <span style="color:#f7e85f">**BBDD + Tablas**</span>

In [6]:
def creacion_bbdd_tablas():
    try:
        # Conectar a MySQL Server
        cnx = mysql.connector.connect(
            user='root',       
            password='AlumnaAdalab',  
            host='127.0.0.1'   
        )
# ------------------------------------------------------------------------------
        if cnx.is_connected():
            print("¡Conectado!")

            mycursor = cnx.cursor()

            # Crear la base de datos si no existe
            mycursor.execute("CREATE DATABASE IF NOT EXISTS BBDD_DataTech_Solutions")
            print("Base de datos 'BBDD_DataTech_Solutions' creada correctamente.")

            # Usar la base de datos
            try:
                mycursor.execute("USE BBDD_DataTech_Solutions")
                print("Base de datos seleccionada correctamente.")
            except mysql.connector.Error as err:
                print("Error al seleccionar la base de datos.")
                print("Código de Error:", err.errno)
                print("SQLSTATE:", err.sqlstate)
                print("Mensaje:", err.msg)
# ------------------------------------------------------------------------------
# Crear la tabla de 'clientes'

            mycursor.execute("""
                CREATE TABLE IF NOT EXISTS clientes (
                    id_cliente VARCHAR (20) PRIMARY KEY,
                    nombre VARCHAR (50),
                    apellido VARCHAR (50), 
                    ciudad VARCHAR (50),        
                    pais  VARCHAR (50)
                );
            """)
            print("Tabla 'clientes' creada.")
# ------------------------------------------------------------------------------
# Crear la tabla de 'productos'

            mycursor.execute("""
                CREATE TABLE IF NOT EXISTS productos (
                    id_producto VARCHAR (20) PRIMARY KEY,
                    precio FLOAT,
                    categoria VARCHAR (200), 
                    descripcion VARCHAR (300)
                );
            """)
            print("Tabla 'productos' creada.")
# ------------------------------------------------------------------------------
# Crear la tabla de 'ventas'

            mycursor.execute("""
                CREATE TABLE IF NOT EXISTS ventas (
                    id_venta VARCHAR (20),
                    id_producto VARCHAR (20),
                    cantidad INT, 
                    fecha DATETIME,
                    id_cliente VARCHAR (20),
                    FOREIGN KEY (id_producto) REFERENCES productos(id_producto),
                    FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente)
                );
            """)
            print("Tabla 'ventas' creada.")
# -----------------------------------------------------------------------------
    except mysql.connector.Error as err:
        print("Se produjo un error al crear la tabla.")
        print("Código de Error:", err.errno)
        print("SQLSTATE:", err.sqlstate)
        print("Mensaje:", err.msg)
# ------------------------------------------------------------------------------
    finally:
        if cnx.is_connected():
            mycursor.close()
            cnx.close()
            print("Conexión MySQL cerrada.")

# ------------------------------------------------------------------------------
# Ejecutar la función para crear la base de datos y tablas
creacion_bbdd_tablas()

¡Conectado!
Base de datos 'BBDD_DataTech_Solutions' creada correctamente.
Base de datos seleccionada correctamente.
Tabla 'clientes' creada.
Tabla 'productos' creada.
Tabla 'ventas' creada.
Conexión MySQL cerrada.


### 🔄 Pasar a <span style="color:#f7e85f">**Lista de Tuplas**</span>

In [7]:
lista_tuplas_clientes = [tuple(row) for row in df_clientes.itertuples(index=False, name=None)]
lista_tuplas_productos = [tuple(row) for row in df_productos.itertuples(index=False, name=None)]
lista_tuplas_ventas = [tuple(row) for row in df_ventas.itertuples(index=False, name=None)]


### ➕ <span style="color:#f7e85f">**Insertar Datos**</span>

* <span style="color:#f7e85f">Conectar</span> con la BBDD

In [8]:
# Conectar a MySQL Server
cnx = mysql.connector.connect(
    user='root',
    password='AlumnaAdalab',
    host='127.0.0.1'
)

mycursor = cnx.cursor()

# Usar la base de datos
try:
    mycursor.execute("USE BBDD_DataTech_Solutions")
    print("Base de datos seleccionada correctamente.")
except mysql.connector.Error as err:
    print("Error al seleccionar la base de datos.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)

Base de datos seleccionada correctamente.


* Tabla <span style="color:#f7e85f">clientes</span>

In [9]:
sql = ("INSERT INTO clientes (id_cliente, nombre, apellido, ciudad, pais) VALUES (%s, %s, %s, %s, %s)")

val = lista_tuplas_clientes

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registros insertados.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

300 registros insertados.


* Tabla <span style="color:#f7e85f">productos</span>

In [10]:
sql = ("INSERT INTO productos (id_producto, precio, categoria, descripcion) VALUES (%s, %s, %s, %s)")

val = lista_tuplas_productos

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registros insertados.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

600 registros insertados.


* Tabla <span style="color:#f7e85f">ventas</span>

In [11]:
sql = ("INSERT INTO ventas (id_venta, id_producto, cantidad, fecha, id_cliente) VALUES (%s, %s, %s, %s, %s)")

val = lista_tuplas_ventas

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registros insertados.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

80000 registros insertados.


* <span style="color:#f7e85f">Cerrar conexión</span> con SQL

In [12]:
# Cerrar el cursor y la conexión a la base de datos
try:
    # Cerrar el cursor si está abierto
    if mycursor:
        mycursor.close()
    
    # cerrar la conexión si está abierta
    if cnx:
        cnx.close()
    
    print("Conexión cerrada correctamente.")
except mysql.connector.Error as err:
    print("Error al cerrar la conexión:", err)

Conexión cerrada correctamente.
